In [ ]:
import os
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras import utils
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
import matplotlib.pyplot as plt

In [ ]:
batch_size = 256
image_size = (300, 300)

train_dataset = image_dataset_from_directory('C:\datasett\\train',
                                             subset='training',
                                             seed = 42,
                                             validation_split=0.1,
                                             batch_size=batch_size,
                                             image_size=image_size)
validation_dataset = image_dataset_from_directory('C:\datasett\\train',
                                                  subset='validation',
                                                  seed = 42,
                                                  validation_split=0.1,
                                                  batch_size=batch_size,
                                                  image_size=image_size)

In [ ]:
class_names = train_dataset.class_names
class_names
plt.figure(figsize = (8,8))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3,3, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis('off')
plt.show()

In [ ]:
test_dataset = image_dataset_from_directory('C:\datasett\\train',
                                            batch_size=batch_size,
                                            image_size=image_size)

In [ ]:
tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = Sequential()
model.add(Conv2D(16, (5,5), padding = 'same',
                 input_shape=(300,300,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(5,5), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (5,5), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(5,5), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(131, activation = 'softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, validation_data=validation_dataset, epochs = 1, verbose = 2)

In [ ]:
scores = model.evaluate(test_dataset, verbose = 1)
print("Доля верных ответов на тестовых данных в процентах:", round(scores[1] * 100, 1))

In [ ]:
plt.plot(history.history['accuracy'],
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], label = 'Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

plt.plot(history.history['loss'],
         label='Ошибка на обучающем наборе')
plt.plot(history.history['val_loss'],
         label='Ошибка на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Ошибка')
plt.legend()
plt.show()

In [ ]:
model.save("fruits_360_model.h5")